In [1]:
from utils import  AverageMeter, LevenshteinDistance, Queue
import cv2
import numpy as np
import build_ddnet_det as ddnet_det
import build_ddnet_clf as ddnet_clf
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from mp_utils import *
import pickle

In [2]:
a = ['/home/giang/Downloads/MICA_data/MICA-Action2020/Subject01Tan/Kinect_1/Annotation_Subject01Tan.txt',
    '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject02Tuan/Kinect_1/Annotation_Subject02Tuan.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject03Viet/Kinect_1/Annotation_Subject03Viet.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject04Binh/Kinect_1/Annotation_Subject04Binh.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject05Cuong/Kinect_1/Annotation_Subject05_Cuong.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject06Luc/Kinect_1/Annotation_Subject06_Luc.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject07HienThanh/Kinect_1/Annotation_Subject07_HienThanh.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject08Lich/Kinect_1/Annotation_Subject08_Lich.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject09Quan/Kinect_1/Annotation_Subject09_Quan.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject10HoangNam/Kinect_1/Annotation_Subject10HoangNam.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject11HaThiOanh/Kinect_1/Annotation_Subject11HaThiOanh.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject12NguyenThiThuyLan/Kinect_1/Annotation_Subject12NguyenThiThuyLan.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject13TranDinhHung/Kinect_1/Annotation_Subject13TranDinhHung.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject14NguyenVietHung/Kinect_1/Annotation_Subject14NguyenVietHung.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject15TaPhuongNam/Kinect_1/Annotation_Subject15TaPhuongNam.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject16DinhQuocHieu/Kinect_1/Annotation_Subject16DinhQuocHieu.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject17CuNgocDai/Kinect_1/Annotation_Subject17CuNgocDai.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject18NguyenTrungHieu/Kinect_1/Annotation_Subject18NguyenTrungHieu.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject19NgoQuangThuy/Kinect_1/Annotation_Subject19NgoQuangThuy.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject20PhamHuuTot/Kinect_1/Annotation_Subject20PhamHuuTot.txt']

In [3]:
import csv

In [4]:
dict_subject=[]
for i in a:
    annotation_file = open(i,'r').readlines()
    dict_of_annotation ={}
    for line in annotation_file:
        line = line.rstrip('\n')
        line = line.split('\t')
        
        action = line[0]
        
        if action!='' :
           
            if (int(action[1:3]))!=0:

                dict_of_annotation[action]= line[1:3]

    for key in dict_of_annotation:
        if int(key[1:3]) !=0:

            if key==' ' or len(dict_of_annotation[key])!=2 or '' in dict_of_annotation[key]:
                print('vcl')
                continue
            
            start_end =  dict_of_annotation[key]
            start = int(start_end[0])
            end = int(start_end[1])
    dict_subject.append(dict_of_annotation)

In [5]:
dict_subject

[{'a01s01e01': ['400', '528'],
  'a01s01e02': ['536', '654'],
  'a01s01e03': ['659', '771'],
  'a02s01e01': ['843', '902'],
  'a02s01e02': ['927', '983'],
  'a02s01e03': ['999', '1054'],
  'a03s01e01': ['1301', '1364'],
  'a03s01e02': ['1376', '1429'],
  'a03s01e03': ['1439', '1489'],
  'a04s01e01': ['1721', '1779'],
  'a04s01e02': ['1804', '1860'],
  'a04s01e03': ['1887', '1946'],
  'a05s01e01': ['2135', '2186'],
  'a05s01e02': ['2203', '2257'],
  'a05s01e03': ['2275', '2333'],
  'a06s01e01': ['2499', '2568'],
  'a06s01e02': ['2592', '2657'],
  'a06s01e03': ['2674', '2743'],
  'a07s01e01': ['3007', '3072'],
  'a07s01e02': ['3095', '3162'],
  'a07s01e03': ['3178', '3240'],
  'a08s01e01': ['3395', '3462'],
  'a08s01e02': ['3486', '3549'],
  'a08s01e03': ['3572', '3632'],
  'a09s01e01': ['4008', '4099'],
  'a09s01e02': ['4121', '4198'],
  'a09s01e03': ['4218', '4291'],
  'a10s01e01': ['4435', '4485'],
  'a10s01e02': ['4506', '4552'],
  'a10s01e03': ['4572', '4623'],
  'a11s01e01': ['4870

In [10]:
precision =[]
recall = []
f1 = []
base_path = '/home/giang/Downloads/MICA_data/'
subject_test = ['01','02', '03', '05', '06', '07', '13', '15', '17', '20']
recognized_path = []
for i in subject_test:
    csv_file = 'continuous_subject{}_mean_16.csv'.format(i)
    recognized_path.append(os.path.join(base_path,csv_file))

In [11]:
recognized_path

['/home/giang/Downloads/MICA_data/continuous_subject01_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject02_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject03_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject05_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject06_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject07_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject13_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject15_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject17_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject20_mean_16.csv']

In [14]:
test_kinect = []
recognized = []
subject_test = [1, 2, 3, 5, 6, 7, 13, 15, 17, 20]
test_mp = []
for index, data in enumerate(subject_test):
    test_kinect.append(a[data-1])
    recognized.append(recognized_path[index])

In [15]:
test_kinect

['/home/giang/Downloads/MICA_data/MICA-Action2020/Subject01Tan/Kinect_1/Annotation_Subject01Tan.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject02Tuan/Kinect_1/Annotation_Subject02Tuan.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject03Viet/Kinect_1/Annotation_Subject03Viet.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject05Cuong/Kinect_1/Annotation_Subject05_Cuong.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject06Luc/Kinect_1/Annotation_Subject06_Luc.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject07HienThanh/Kinect_1/Annotation_Subject07_HienThanh.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject13TranDinhHung/Kinect_1/Annotation_Subject13TranDinhHung.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject15TaPhuongNam/Kinect_1/Annotation_Subject15TaPhuongNam.txt',
 '/home/giang/Downloads/MICA_data/MICA-Action2020/Subject17CuNgocDai/Kinect_1/Annotation_Subject17CuNgocDai.txt',
 '/home/giang

In [16]:
recognized

['/home/giang/Downloads/MICA_data/continuous_subject01_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject02_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject03_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject05_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject06_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject07_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject13_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject15_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject17_mean_16.csv',
 '/home/giang/Downloads/MICA_data/continuous_subject20_mean_16.csv']

In [17]:
def cal_segment_score(detected_file, ground_truth):
    data = pd.read_csv(detected_file)
    predicted_class = data['Action'].tolist()
    #predicted_class = predicted_class[:-1]
    levenshtein_distance = LevenshteinDistance(ground_truth, predicted_class)
    levenshtein_accuracy = 1 - (levenshtein_distance / len(ground_truth))
    return levenshtein_accuracy

In [23]:
subject_test1 = ['1','2', '3', '5', '6', '7', '13', '15', '17', '20']
subject_test = []
for i in subject_test1:
    a = 'subject{}'.format(i)
    subject_test.append(a)
    

In [26]:
import pandas as pd

In [19]:
data_test = {}
list_subject = [ 'subject{}'.format(i) for i in range(1,21,1)]
for i in list_subject:
    data_test[i] = {}
    data_test[i]['pose'] = []
    data_test[i]['label'] = []
    data_test[i]['segment'] = [] 

In [20]:

base_path = '/home/giang/Downloads/MICA_Action3d/Full_data/'
for index,i  in enumerate(dict_subject):
    for key in i:
        a = "a%02i_s%02i_e%02i_skeleton3D.txt" %(int(key[1:3]),int(key[4:6]),int(key[7:9]) )
        path = os.path.join(base_path,a)
        name = path.rpartition('/')[-1]
        pose_id = int(name[1:3])
        pose_skeletal = np.loadtxt(path).astype('float32')
        pose_skeletal = np.reshape(pose_skeletal, (-1,20,3))
        #print(name)
        #print( pose_skeletal.shape[0])
        #print(dict_of_annotation[key])
        #print(int(dict_of_annotation[key][-1]) - int(dict_of_annotation[key][0] ))
        #print('================')
    
        data_test[list_subject[index]]['pose'].append(pose_skeletal)
        data_test[list_subject[index]]['label'].append(pose_id)
        data_test[list_subject[index]]['segment'].append(i[key])
    

In [13]:
import pandas as pd

In [14]:
leivenshtein_acc = []
for i in range(10):

    csv_file = file[i]
    subject = 'subject{}'.format(int(file[i][-6:-4]))
    ground_truth = data_test[subject]['label']
    levenshtein_accuracy = cal_segment_score(csv_file, ground_truth)
    leivenshtein_acc.append(levenshtein_accuracy)

NameError: name 'file' is not defined

In [28]:
leivenshtein_acc

[0.9,
 0.9166666666666666,
 0.95,
 0.9833333333333333,
 0.9166666666666666,
 0.9322033898305084,
 0.9333333333333333,
 0.9666666666666667,
 0.8166666666666667,
 0.95]

In [29]:
from statistics import mean
print(mean(leivenshtein_acc))

0.9265536723163842


In [32]:
def get_predicted(csv_file):
    data = pd.read_csv(csv_file)
    

        
    p_start = []
    p_end = []
    p_label = []
    for i in range(len(data)):
        p_start.append(data['Start frame'][i])
        p_end.append(data['End frame'][i])
        p_label.append(data['Action'][i])

        
    return p_start, p_end, p_label

In [33]:
def get_ground_truth(file):
    y_start = []
    y_end = []
    y_label = []
    annotation_file = open(file,'r').readlines()
    dict_of_annotation ={}
    for line in annotation_file:
        line = line.rstrip('\n')
        line = line.split('\t')
        action = line[0]
        if action != '':

            dict_of_annotation[action]= line[1:3]
    for key in dict_of_annotation:
        if int(key[1:3]) !=0:

            if key==' ' or len(dict_of_annotation[key])!=2 or '' in dict_of_annotation[key]:
                print('vcl')
                continue
            start_end =  dict_of_annotation[key]
            start = int(start_end[0])
            end = int(start_end[1])
            y_start.append(start)
            y_end.append(end)
            y_label.append(int(key[1:3]))
    return y_start, y_end, y_label

In [34]:
def f_score(recognized, ground_truth, overlap):
    p_start, p_end, p_label = get_predicted(recognized)
    y_start, y_end, y_label = get_ground_truth(ground_truth)

    tp = 0
    fp = 0

    hits = np.zeros(len(y_label))

    for j in range(len(p_label)):
        intersection = np.minimum(p_end[j], y_end) - np.maximum(p_start[j], y_start)
        union = np.maximum(p_end[j], y_end) - np.minimum(p_start[j], y_start)
        IoU = (1.0*intersection / union)*([p_label[j] == y_label[x] for x in range(len(y_label))])
        # Get the best scoring segment
        idx = np.array(IoU).argmax()

        if IoU[idx] >= overlap and not hits[idx]:
            tp += 1
            hits[idx] = 1
        else:
            fp += 1
    fn = len(y_label) - sum(hits)
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    f1 = 2.0 * (precision*recall) / (precision+recall)
    return float(precision), float(recall), float(f1)

In [43]:
# kinect
precision = []
recall = []
f1_score = []
overlap = [0.25, 0.4, 0.5 ]

for i in overlap:
    print('overlap {}'.format(i))
    for j in range(10):
    
        precision1,recall1, f11 = f_score(recognized[j], test_kinect[j], i)

        precision.append(precision1)
        recall.append(recall1)
        f1_score.append(f11)
        print('precision@{}: {:.2f}, recall@{}: {:.2f}, f1 score:{:.2f}'.format(i,precision1,i,recall1, f11))
    print('==========================')

overlap 0.25
precision@0.25: 0.71, recall@0.25: 0.78, f1 score:0.75
precision@0.25: 0.88, recall@0.25: 0.93, f1 score:0.90
precision@0.25: 0.90, recall@0.25: 0.95, f1 score:0.93
precision@0.25: 0.90, recall@0.25: 0.92, f1 score:0.91
precision@0.25: 0.97, recall@0.25: 0.95, f1 score:0.96
precision@0.25: 0.87, recall@0.25: 0.93, f1 score:0.90
precision@0.25: 0.97, recall@0.25: 0.93, f1 score:0.95
precision@0.25: 0.97, recall@0.25: 1.00, f1 score:0.98
precision@0.25: 0.83, recall@0.25: 0.95, f1 score:0.88
precision@0.25: 0.92, recall@0.25: 0.97, f1 score:0.94
overlap 0.4
precision@0.4: 0.47, recall@0.4: 0.52, f1 score:0.49
precision@0.4: 0.80, recall@0.4: 0.85, f1 score:0.82
precision@0.4: 0.76, recall@0.4: 0.80, f1 score:0.78
precision@0.4: 0.77, recall@0.4: 0.78, f1 score:0.78
precision@0.4: 0.83, recall@0.4: 0.82, f1 score:0.82
precision@0.4: 0.67, recall@0.4: 0.71, f1 score:0.69
precision@0.4: 0.81, recall@0.4: 0.78, f1 score:0.80
precision@0.4: 0.73, recall@0.4: 0.75, f1 score:0.74
p

In [44]:
# kinect
print(mean(precision[0:10]))
print(mean(precision[10:20]))
print(mean(precision[20:30]))
print('===============')
print(mean(recall[0:10]))
print(mean(recall[10:20]))
print(mean(recall[20:30]))
print('================')
print(mean(f1_score[0:10]))
print(mean(f1_score[10:20]))
print(mean(f1_score[20:30]))

0.891262108309638
0.7202895369401143
0.54314376202823
0.9315536723163842
0.7511864406779661
0.5660169491525424
0.9104369728283355
0.7350011131263738
0.5540346219167315


In [18]:
test_kinect = []
recognized = []
subject_test = [1, 2, 3, 5, 6, 7, 13, 15, 17, 20]
test_mp = []
for i in subject_test:
    print(a[i-1])
    test_kinect.append(a[i-1])


a
2
0
s
2
0
s
e
e
n


In [19]:
test_kinect

['a', '2', '0', 's', '2', '0', 's', 'e', 'e', 'n']

In [47]:
recognized = file

In [48]:
recognized

['/home/giang/Downloads/MICA_data/mediapipe_detected_result_sub01.csv',
 '/home/giang/Downloads/MICA_data/mediapipe_detected_result_sub02.csv',
 '/home/giang/Downloads/MICA_data/mediapipe_detected_result_sub03.csv',
 '/home/giang/Downloads/MICA_data/mediapipe_detected_result_sub05.csv',
 '/home/giang/Downloads/MICA_data/mediapipe_detected_result_sub06.csv',
 '/home/giang/Downloads/MICA_data/mediapipe_detected_result_sub07.csv',
 '/home/giang/Downloads/MICA_data/mediapipe_detected_result_sub13.csv',
 '/home/giang/Downloads/MICA_data/mediapipe_detected_result_sub15.csv',
 '/home/giang/Downloads/MICA_data/mediapipe_detected_result_sub17.csv',
 '/home/giang/Downloads/MICA_data/mediapipe_detected_result_sub20.csv']